# London Open Trend Follow — Backtesting Analysis

This notebook backtests a simple intraday trend-following strategy based on the direction of the last London session candle before 08:00 UTC.

## Strategy Overview

At the close of the **07:45 UTC** M15 candle (last bar of the 07:xx hour), determine the session direction and enter on the next bar. Exit intraday — no overnight positions, no overnight fees.

### Entry Rule
- **LONG**: 07:45 UTC M15 candle closes bullish (close > open) → enter at 08:00 UTC open
- **SHORT**: 07:45 UTC M15 candle closes bearish (close < open) → enter at 08:00 UTC open

### Exit Rule
- Close position at **17:00 UTC open** (configurable)
- One trade per day maximum
- No stop loss / take profit — pure time-based exit (starting point; SL/TP added in optimisation)

### Research Background
Grid search over 258 trading days of EUR/USD H1 2025 data showed:

| Entry | Exit | Win Rate | Avg P&L | Total P&L |
|-------|------|----------|---------|-----------|
| 08:00 UTC (after 07:45 signal) | 17:00 UTC | ~52–63% | +5–16 pips | +1,200–4,000 pips |

The 07:45 signal + 17:00 exit was the single best combination found.

### Files
- Data: `data/historical/EUR_USD/EUR_USD_M15_20250101_20251231.csv`

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print('✓ Imports successful')

## 2. Configuration

In [ ]:
MAJOR_PAIRS = ['EUR_USD', 'GBP_USD', 'USD_JPY', 'USD_CHF', 'USD_CAD', 'AUD_USD', 'NZD_USD']
GRANULARITY  = 'M15'
FROM_DATE    = '20250101'
TO_DATE      = '20251231'

INITIAL_CASH = 10_000
COMMISSION   = 0.0001   # ~1 pip round-trip

DATA_DIR = Path('../data/historical')

print(f'Pairs      : {", ".join(MAJOR_PAIRS)}')
print(f'Timeframe  : {GRANULARITY}')
print(f'Period     : {FROM_DATE} – {TO_DATE}')
print(f'Capital    : ${INITIAL_CASH:,.0f}')
print()
for pair in MAJOR_PAIRS:
    p = DATA_DIR / pair / f'{pair}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'
    print(f'  {pair}: {"✓" if p.exists() else "✗ MISSING"}  {p}')

## 3. Load Data

In [ ]:
def load_data(path: Path) -> pd.DataFrame:
    """Load OANDA CSV, return DataFrame formatted for backtesting.py."""
    df = pd.read_csv(path, comment='#', parse_dates=['time'], index_col='time')
    df.index = pd.to_datetime(df.index, utc=True)
    df.columns = [c.capitalize() for c in df.columns]
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]


datasets = {}
for pair in MAJOR_PAIRS:
    path = DATA_DIR / pair / f'{pair}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'
    datasets[pair] = load_data(path)
    df = datasets[pair]
    print(f'{pair}: {len(df):,} candles  ({df.index.min().date()} → {df.index.max().date()})')

print(f'\n✓ Loaded {len(datasets)} pairs')

## 4. Exploratory Analysis

Understand hourly volatility and directional bias before writing any strategy code.

In [ ]:
def hourly_stats(pair, df_bt):
    """Return raw hourly stats DataFrame from a backtesting.py-formatted DataFrame."""
    raw = df_bt.copy().reset_index()
    raw.columns = [c.lower() for c in raw.columns]
    raw['hour']  = raw['time'].dt.hour
    raw['range'] = (raw['high'] - raw['low']) * 10_000
    raw['ret']   = (raw['close'] - raw['open']) * 10_000
    raw['bull']  = raw['ret'] > 0
    return raw.groupby('hour').agg(
        avg_range=('range', 'mean'),
        avg_ret  =('ret',   'mean'),
        win_rate =('bull',  'mean'),
    ).round(2)


fig, axes = plt.subplots(len(MAJOR_PAIRS), 3, figsize=(16, 3 * len(MAJOR_PAIRS)))

for row, pair in enumerate(MAJOR_PAIRS):
    hourly = hourly_stats(pair, datasets[pair])
    hrs = hourly.loc[6:21]

    axes[row, 0].bar(hrs.index, hrs['avg_range'], color='steelblue')
    axes[row, 0].axvline(7, color='red', linestyle='--', linewidth=1)
    axes[row, 0].set_title(f'{pair} — Avg Range (pips)', fontsize=9)
    axes[row, 0].set_xlabel('Hour UTC', fontsize=8)

    colors = ['green' if v > 0 else 'red' for v in hrs['avg_ret']]
    axes[row, 1].bar(hrs.index, hrs['avg_ret'], color=colors)
    axes[row, 1].axhline(0, color='black', linewidth=0.8)
    axes[row, 1].axvline(7, color='red', linestyle='--', linewidth=1)
    axes[row, 1].set_title(f'{pair} — Avg Direction (pips)', fontsize=9)
    axes[row, 1].set_xlabel('Hour UTC', fontsize=8)

    axes[row, 2].bar(hrs.index, hrs['win_rate'] * 100, color='purple', alpha=0.7)
    axes[row, 2].axhline(50, color='black', linewidth=0.8, linestyle='--')
    axes[row, 2].axvline(7, color='red', linestyle='--', linewidth=1)
    axes[row, 2].set_title(f'{pair} — Bullish Win Rate (%)', fontsize=9)
    axes[row, 2].set_xlabel('Hour UTC', fontsize=8)

plt.suptitle('Major Pairs M15 2025 — Hourly Statistics  (red dashed = 07:00 UTC)', fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

### 07:00 Candle — Signal Quality Check

How well does the 07:00 candle predict the rest of the day?

In [ ]:
def signal_analysis(pair, df_bt):
    """Compute per-day signal stats for a pair."""
    raw = df_bt.copy().reset_index()
    raw.columns = [c.lower() for c in raw.columns]
    raw['hour']   = raw['time'].dt.hour
    raw['minute'] = raw['time'].dt.minute
    raw['date']   = raw['time'].dt.date
    raw['ret']    = (raw['close'] - raw['open']) * 10_000

    rows = []
    for date, day in raw.groupby('date'):
        day = day.sort_values('time')
        # Signal: last M15 bar of hour 7 (07:45), entry fills at 08:00 open
        sig   = day[(day['hour'] == 7) & (day['minute'] == 45)]
        entry = day[day['hour'] == 8]
        exit_ = day[day['hour'] == 17]
        if len(sig) == 0 or len(entry) == 0 or len(exit_) == 0:
            continue
        direction  = 1 if sig.iloc[-1]['ret'] > 0 else -1
        trade_pnl  = direction * (exit_.iloc[0]['open'] - entry.iloc[0]['open']) * 10_000
        rows.append({'date': date, 'dow': pd.Timestamp(date).day_name(),
                     'signal_dir': direction, 'trade_pnl': trade_pnl, 'win': trade_pnl > 0})
    return pd.DataFrame(rows)


# Build summary across all pairs
summary_rows = []
all_signals = {}
for pair in MAJOR_PAIRS:
    sig = signal_analysis(pair, datasets[pair])
    all_signals[pair] = sig
    summary_rows.append({
        'Pair':          pair,
        'Trading Days':  len(sig),
        'Win Rate (%)':  f"{sig['win'].mean()*100:.1f}%",
        'Avg P&L (pips)': round(sig['trade_pnl'].mean(), 1),
        'Total P&L (pips)': round(sig['trade_pnl'].sum(), 1),
    })

summary = pd.DataFrame(summary_rows).set_index('Pair')
print('07:45 UTC Signal Quality — 08:00 entry → 17:00 exit\n')
display(summary)

# Cumulative P&L for all pairs
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for pair in MAJOR_PAIRS:
    sig = all_signals[pair]
    axes[0].plot(range(len(sig)), sig['trade_pnl'].cumsum(), label=pair, linewidth=1.5)
axes[0].axhline(0, color='black', linewidth=0.8, linestyle='--')
axes[0].set_title('Cumulative P&L by Pair (pips)  [08:00→17:00 UTC, no sizing]')
axes[0].set_xlabel('Trade #')
axes[0].set_ylabel('Cumulative pips')
axes[0].legend(fontsize=8)

# Win rates
win_rates = [all_signals[p]['win'].mean() * 100 for p in MAJOR_PAIRS]
colors = ['green' if w > 50 else 'red' for w in win_rates]
axes[1].bar(MAJOR_PAIRS, win_rates, color=colors, edgecolor='black', alpha=0.8)
axes[1].axhline(50, color='black', linewidth=0.8, linestyle='--', label='50%')
axes[1].set_title('Win Rate by Pair (%)  [07:45 signal → 17:00 exit]')
axes[1].set_ylabel('Win Rate (%)')
axes[1].legend()

plt.tight_layout()
plt.show()

### Day-of-Week Breakdown

In [ ]:
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

fig, axes = plt.subplots(2, 4, figsize=(18, 7))
axes = axes.flatten()

for idx, pair in enumerate(MAJOR_PAIRS):
    sig = all_signals[pair]
    dow_avg = sig.groupby('dow')['trade_pnl'].mean().reindex(dow_order)
    colors  = ['green' if v > 0 else 'red' for v in dow_avg]
    axes[idx].bar(dow_avg.index, dow_avg.values, color=colors, edgecolor='black', alpha=0.8)
    axes[idx].axhline(0, color='black', linewidth=0.8)
    axes[idx].set_title(f'{pair}', fontsize=10)
    axes[idx].set_ylabel('Avg P&L (pips)', fontsize=8)
    axes[idx].set_xticklabels([d[:3] for d in dow_order], fontsize=8)

axes[-1].set_visible(False)   # hide unused subplot
plt.suptitle('Avg P&L by Day of Week — All Major Pairs (pips)', fontsize=13)
plt.tight_layout()
plt.show()

# Summary table: avg P&L per pair × day
dow_table = pd.DataFrame({
    pair: all_signals[pair].groupby('dow')['trade_pnl'].mean().reindex(dow_order).round(1)
    for pair in MAJOR_PAIRS
})
print('\nAvg P&L per day × pair (pips):')
display(dow_table)

## 5. Strategy Implementation

Three variants:
- **Base**: enter at 08:00 UTC (after 07:00 candle closes), exit at `exit_hour` open
- **DayFilter**: same, but only trade Monday and Wednesday (best days from analysis)
- **WithSL**: adds a stop loss and take profit in pips

In [ ]:
class LondonOpenTrendFollow(Strategy):
    """
    Enter at 08:00 UTC open in the direction of the 07:45 UTC M15 candle.
    Exit at exit_hour UTC open (time-based, no SL/TP).
    One trade per day.
    """
    exit_hour = 17   # UTC hour to close position (executes at that hour's open)

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]
        bar_hour = bar_time.hour

        # --- Exit: close at exit_hour open (first bar of that hour) ---
        if bar_hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        # --- Entry signal: 07:45 UTC M15 candle (last bar of hour 7) ---
        # self.buy() / self.sell() execute at next bar's open (08:00 UTC)
        if bar_hour == 7 and bar_time.minute == 45:
            o = self.data.Open[-1]
            c = self.data.Close[-1]
            if c > o:
                self.buy()
            elif c < o:
                self.sell()


class LondonOpenDayFilter(Strategy):
    """
    Same as LondonOpenTrendFollow but only trades Monday (0) and Wednesday (2).
    """
    exit_hour = 17
    trade_days = (0, 2)   # Mon=0, Wed=2

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]
        bar_hour = bar_time.hour

        if bar_hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_hour == 7 and bar_time.minute == 45 and bar_time.weekday() in self.trade_days:
            o = self.data.Open[-1]
            c = self.data.Close[-1]
            if c > o:
                self.buy()
            elif c < o:
                self.sell()


class LondonOpenWithSL(Strategy):
    """
    London Open + stop loss and take profit in pips.
    Still exits at exit_hour if neither SL nor TP has been hit.
    """
    exit_hour  = 17
    sl_pips    = 20   # stop loss distance in pips
    tp_pips    = 40   # take profit distance in pips

    def init(self):
        pass

    def next(self):
        bar_time = self.data.index[-1]
        bar_hour = bar_time.hour

        if bar_hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        if bar_hour == 7 and bar_time.minute == 45:
            o = self.data.Open[-1]
            c = self.data.Close[-1]
            pip = 0.0001
            sl  = self.sl_pips * pip
            tp  = self.tp_pips * pip
            if c > o:
                self.buy(sl=c - sl, tp=c + tp)
            elif c < o:
                self.sell(sl=c + sl, tp=c - tp)


print('✓ Strategy classes defined')
print('  - LondonOpenTrendFollow : base strategy')
print('  - LondonOpenDayFilter   : Mon & Wed only')
print('  - LondonOpenWithSL      : with stop loss / take profit')

## 6. Backtest — Base Strategy

In [ ]:
all_results = {}   # {pair: {strategy_label: stats}}

for pair in MAJOR_PAIRS:
    df = datasets[pair]
    all_results[pair] = {}

    bt = Backtest(df, LondonOpenTrendFollow, cash=INITIAL_CASH, commission=COMMISSION)
    all_results[pair]['Base']     = bt.run()

    bt = Backtest(df, LondonOpenDayFilter, cash=INITIAL_CASH, commission=COMMISSION)
    all_results[pair]['DayFilter'] = bt.run()

    bt = Backtest(df, LondonOpenWithSL, cash=INITIAL_CASH, commission=COMMISSION)
    all_results[pair]['WithSL']   = bt.run()

    print(f'✓ {pair}: Base={all_results[pair]["Base"]["Return [%]"]:.1f}%  '
          f'DayFilter={all_results[pair]["DayFilter"]["Return [%]"]:.1f}%  '
          f'WithSL={all_results[pair]["WithSL"]["Return [%]"]:.1f}%')

print('\n✓ All backtests complete')

## 7. Backtest — Day Filter (Mon & Wed only)

In [ ]:
# Results are already computed in the cell above — see Section 9 for full comparison

## 8. Backtest — With Stop Loss & Take Profit

In [ ]:
# Results are already computed in the cell above — see Section 9 for full comparison

## 9. Strategy Comparison

In [ ]:
STRATEGIES = ['Base', 'DayFilter', 'WithSL']
METRICS = ['Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Max. Drawdown [%]', '# Trades']

for metric in METRICS:
    table = pd.DataFrame(
        {strat: {pair: round(all_results[pair][strat][metric], 2) for pair in MAJOR_PAIRS}
         for strat in STRATEGIES}
    )
    print(f'\n── {metric} ──')
    display(table)

# Equity curves — Base strategy, all pairs
fig = go.Figure()
for pair in MAJOR_PAIRS:
    eq = all_results[pair]['Base']['_equity_curve']
    fig.add_trace(go.Scatter(x=eq.index, y=eq['Equity'], name=pair, line=dict(width=1.5)))

fig.add_hline(y=INITIAL_CASH, line_dash='dash', line_color='gray', annotation_text='Initial capital')
fig.update_layout(
    title='London Open Base Strategy — Equity Curves (All Major Pairs)',
    xaxis_title='Date', yaxis_title='Equity ($)',
    height=500, hovermode='x unified'
)
fig.show()

## 10. Trade Analysis — Base Strategy

In [ ]:
trade_summary = []
for pair in MAJOR_PAIRS:
    trades = all_results[pair]['Base']['_trades'].copy()
    if len(trades) == 0:
        continue
    wins   = trades[trades['PnL'] > 0]
    losses = trades[trades['PnL'] < 0]
    dur_h  = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds() / 3600
    trade_summary.append({
        'Pair':          pair,
        '# Trades':      len(trades),
        'Win Rate (%)':  f"{len(wins)/len(trades)*100:.1f}%",
        'Avg Win ($)':   f"${wins['PnL'].mean():.2f}" if len(wins) else 'N/A',
        'Avg Loss ($)':  f"${losses['PnL'].mean():.2f}" if len(losses) else 'N/A',
        'Avg Duration (h)': round(dur_h.mean(), 1),
        'Final Equity ($)': f"${all_results[pair]['Base']['Equity Final [$]']:,.2f}",
    })

print('Base Strategy — Trade Summary Across All Major Pairs\n')
display(pd.DataFrame(trade_summary).set_index('Pair'))

## 11. Parameter Optimisation — Exit Hour

Test exit hours from 14:00 to 21:00 UTC to find the best intraday close time.

In [ ]:
exit_results = []

for exit_h in range(14, 22):
    bt = Backtest(df, LondonOpenTrendFollow, cash=INITIAL_CASH, commission=COMMISSION)
    s = bt.run(exit_hour=exit_h)
    exit_results.append({
        'exit_hour': exit_h,
        'n_trades':   int(s['# Trades']),
        'win_rate':   round(s['Win Rate [%]'], 1),
        'return_pct': round(s['Return [%]'], 2),
        'sharpe':     round(s['Sharpe Ratio'], 2),
        'max_dd':     round(s['Max. Drawdown [%]'], 2),
        'final_eq':   round(s['Equity Final [$]'], 2),
    })

exit_df = pd.DataFrame(exit_results).set_index('exit_hour')
print('Exit hour optimisation:')
display(exit_df)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(exit_df.index, exit_df['win_rate'], marker='o', color='steelblue')
axes[0].set_title('Win Rate (%) by Exit Hour')
axes[0].set_xlabel('Exit Hour (UTC)')
axes[0].axhline(50, color='red', linestyle='--')

axes[1].plot(exit_df.index, exit_df['return_pct'], marker='o', color='green')
axes[1].set_title('Total Return (%) by Exit Hour')
axes[1].set_xlabel('Exit Hour (UTC)')

axes[2].plot(exit_df.index, exit_df['sharpe'], marker='o', color='darkorange')
axes[2].set_title('Sharpe Ratio by Exit Hour')
axes[2].set_xlabel('Exit Hour (UTC)')

plt.suptitle('Impact of Exit Hour on Strategy Performance', fontsize=12)
plt.tight_layout()
plt.show()

## 12. SL/TP Optimisation

In [ ]:
print('Optimising SL and TP pips for LondonOpenWithSL...')
print('(this may take ~1 minute)')

bt_sltp = Backtest(df, LondonOpenWithSL, cash=INITIAL_CASH, commission=COMMISSION)
stats_sltp_opt = bt_sltp.optimize(
    sl_pips=range(10, 41, 10),
    tp_pips=range(20, 81, 10),
    exit_hour=range(15, 20),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.tp_pips > p.sl_pips,
    return_heatmap=False,
)

print('\n' + '=' * 70)
print('OPTIMISED SL/TP RESULTS')
print('=' * 70)
print(stats_sltp_opt)
print(f"\nBest SL  : {stats_sltp_opt._strategy.sl_pips} pips")
print(f"Best TP  : {stats_sltp_opt._strategy.tp_pips} pips")
print(f"Best exit: {stats_sltp_opt._strategy.exit_hour}:00 UTC")

## 13. Interactive Backtest Chart

In [ ]:
# Interactive chart for a single pair (EUR_USD as reference)
# Change CHART_PAIR to inspect any pair
CHART_PAIR = 'EUR_USD'
bt_chart = Backtest(datasets[CHART_PAIR], LondonOpenTrendFollow, cash=INITIAL_CASH, commission=COMMISSION)
bt_chart.run()
bt_chart.plot(open_browser=True)

<cell_type>markdown</cell_type>## 14. Conclusions

### Key Findings

| Question | Answer |
|----------|--------|
| Which pairs respond best to the 07:00 UTC signal? | *(fill after running)* |
| Which pairs have win rate > 50%? | *(fill after running)* |
| Does the Day Filter improve results across pairs? | *(fill after running)* |
| Does adding SL/TP help or hurt across pairs? | *(fill after running)* |
| Which day of the week is strongest per pair? | *(fill after running)* |

### Strategy Strengths
- One trade per day — simple to manage
- Fully exits intraday — zero overnight fee risk
- Signal is objective (candle direction, no indicator lag)
- London open is the highest-liquidity window for EUR/USD and GBP/USD

### Strategy Weaknesses
- No trend filter — trades every day regardless of macro context
- No volatility filter — treats a 2-pip candle the same as a 20-pip candle
- USD/JPY pip value differs (0.01 vs 0.0001) — commission and SL/TP pips need adjusting for JPY pairs
- In-sample only — **must validate on out-of-sample data** before live trading

### Recommended Next Steps
1. **Out-of-sample test**: run on 2024 data for all pairs to check for overfitting
2. **JPY pip correction**: adjust pip multiplier for USD_JPY (use 0.01 instead of 0.0001)
3. **Volatility filter**: only trade when the 07:00 candle range > X pips
4. **Per-pair optimisation**: run exit hour and SL/TP optimisation for each pair separately
5. **Portfolio view**: combine the best-performing pairs into a single portfolio backtest

---
**Reminder**: In-sample backtest results are optimistic. Past performance does not guarantee future results.